## Домашнее задание

___Не успел закончить (был в коммандировке неделю), оставляю зашлушку за выходные до 25.04 закончу.___

### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.


### Задание 2.

найти фичи с наибольшей значимостью, и вывести их


### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

## Импорты

In [107]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [22]:
df = pd.read_pickle('../Lesson_01/lesson_01_preprocessing.pkl')
df.head()

,id,label,tweet,clear_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,user when father is dysfunctional and is so se...,"[user, when, father, is, dysfunctional, and, i...","[user, father, dysfunctional, selfish, drags, ...","[user, father, dysfunct, selfish, drag, kid, d...","[user, father, dysfunct, selfish, drag, kid, d..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit can use cause...,"[user, user, thanks, for, lyft, credit, can, u...","[user, user, thanks, lyft, credit, use, cause,...","[user, user, thank, lyft, credit, use, caus, o...","[user, user, thank, lyft, credit, use, caus, o..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguid, societi, motiv]"


Проверим на пропуски столбец ``'label'`` и отсутствующие заполним ``0`` (на этой итерации)


Затем построим классификатор и на его основании предскажем метки твитов с пропущенными метками

In [23]:
df['label'].unique()

array([ 0.,  1., nan])

In [24]:
df['label'].value_counts()

0.0    29720
1.0     2242
Name: label, dtype: int64

In [25]:
df['label'].fillna(0, inplace=True)

In [26]:
df['label'].astype(int).value_counts()

0    46917
1     2242
Name: label, dtype: int64

### Выделим метки и разделим на train и test

In [93]:
X, y = df['tweet_stemmed'], df['label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

X_train = [" ".join(x) for x in X_train]
X_test = [" ".join(x) for x in X_test]

### Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой


Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [80]:
corpus = [x for l in df['tweet_stemmed'] for x in l]
print(len(corpus))
corpus[:2]

403016


['user', 'father']

In [82]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('user', 27137),
 ('love', 5016),
 ('day', 4423),
 ('happi', 3285),
 ('amp', 2710),
 ('thank', 2407),
 ('time', 1957),
 ('get', 1943),
 ('like', 1828),
 ('life', 1797)]

In [110]:
vec = CountVectorizer(ngram_range=(1, 1), binary=False)
bow = vec.fit_transform(X_train)

clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(vec.transform(X_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     15793
         1.0       0.33      0.58      0.42       430

    accuracy                           0.96     16223
   macro avg       0.66      0.77      0.70     16223
weighted avg       0.97      0.96      0.96     16223



In [111]:
vec = CountVectorizer(ngram_range=(1, 2), binary=False)
bow = vec.fit_transform(X_train)

clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(vec.transform(X_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98     15805
         1.0       0.34      0.61      0.43       418

    accuracy                           0.96     16223
   macro avg       0.66      0.79      0.71     16223
weighted avg       0.97      0.96      0.96     16223



In [113]:
vec = TfidfVectorizer(ngram_range=(1, 2))
bow = vec.fit_transform(X_train)

clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

pred = clf.predict(vec.transform(X_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98     16052
         1.0       0.13      0.60      0.22       171

    accuracy                           0.96     16223
   macro avg       0.56      0.78      0.60     16223
weighted avg       0.99      0.96      0.97     16223



In [114]:
vec.idf_

array([10.30373964, 10.70920474, 10.70920474, ..., 10.70920474,
       10.70920474, 10.70920474])

## Получить наиболее значимые фичи

Не помоню - посмотри в инете

## Попробуем сетку

In [122]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout

In [125]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [126]:
train_data = train_data.batch(10)
test_data = test_data.batch(10)

In [129]:
# AUTOTUNE = tf.data.AUTOTUNE
# train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
# test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

In [130]:
def custom_standartization(input_data):
    return input_data

In [ ]:
vocab_size = 10000
seq_len = 100

vectorize_layer = TextVectorization(
                    )